In [1]:
import _aggregate_agency
import _report_utils
import _section1_utils as section1
import _section2_utils as section2
import geopandas as gpd
import pandas as pd

In [2]:
import altair as alt
import yaml

with open("readable.yml") as f:
    readable_dict = yaml.safe_load(f)

# Color Palette
with open("color_palettes.yml") as f:
    color_dict = yaml.safe_load(f)

import _report_utils
from IPython.display import HTML, Markdown, display, display_html

In [3]:
import yaml
from shared_utils import catalog_utils, rt_dates, rt_utils
with open("readable.yml") as f:
    readable_dict = yaml.safe_load(f)
GTFS_DATA_DICT = catalog_utils.get_catalog("gtfs_analytics_data")

In [4]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [5]:
organization_name = "Marin County Transit District"

In [14]:
def load_operator_metrics(name:str)->pd.DataFrame:
    """
    Load dataframe with the total scheduled service hours 
    a transit operator.
    """
    url = f"{GTFS_DATA_DICT.digest_tables.dir}{GTFS_DATA_DICT.digest_tables.operator_metrics}.parquet"

    df = pd.read_parquet(url,
    filters=[[(("organization_name", "==", name))]])
    
    # Rename dataframe
    df.columns = df.columns.map(_report_utils.replace_column_names)
    
    df["ruler_100_pct"] = 100
    
    df["ruler_for_vp_per_min"] = 2
    return df

In [15]:
df = load_operator_metrics(organization_name)

In [24]:
df.shape

(19, 8)

In [17]:
agency_spatial_chart = (
        section2.simple_bar_chart(
            df,
            "Spatial Accuracy (All Routes)",
            "ruler_100_pct",
            readable_dict["agency_spatial_accuracy"]["title"],
            readable_dict["spatial_accuracy_graph"]["subtitle"],
            color_dict["spatial_accuracy_domain"],
            color_dict["spatial_accuracy_range"]
        )
    )
    

In [18]:
agency_spatial_chart

alt.LayerChart(...)

In [19]:
agency_vp_chart = (
        (
            section2.simple_bar_chart(
                df,
                "VP per Minute (All Routes)",
                "ruler_for_vp_per_min",
                readable_dict["agency_vp_per_min_graph"]["title"],
                readable_dict["vp_per_min_graph"]["subtitle"],
                color_dict["vp_domain"],
                color_dict["vp_range"]
            )
        )
    )

In [20]:
agency_vp_chart

alt.LayerChart(...)

### Look at City of Visalia


In [26]:
df2 = load_operator_metrics("City of Visalia")

In [27]:
(
        (
            section2.simple_bar_chart(
                df2,
                "VP per Minute (All Routes)",
                "ruler_for_vp_per_min",
                readable_dict["agency_vp_per_min_graph"]["title"],
                readable_dict["vp_per_min_graph"]["subtitle"],
                color_dict["vp_domain"],
                color_dict["vp_range"]
            )
        )
    )

alt.LayerChart(...)

In [28]:
(
        section2.simple_bar_chart(
            df2,
            "Spatial Accuracy (All Routes)",
            "ruler_100_pct",
            readable_dict["agency_spatial_accuracy"]["title"],
            readable_dict["spatial_accuracy_graph"]["subtitle"],
            color_dict["spatial_accuracy_domain"],
            color_dict["spatial_accuracy_range"]
        )
    )
    

alt.LayerChart(...)

### Orange County Transportation Authority

In [29]:
df3 = load_operator_metrics("Orange County Transportation Authority")

In [30]:
(
        (
            section2.simple_bar_chart(
                df3,
                "VP per Minute (All Routes)",
                "ruler_for_vp_per_min",
                readable_dict["agency_vp_per_min_graph"]["title"],
                readable_dict["vp_per_min_graph"]["subtitle"],
                color_dict["vp_domain"],
                color_dict["vp_range"]
            )
        )
    )

alt.LayerChart(...)

In [31]:
(
        section2.simple_bar_chart(
            df3,
            "Spatial Accuracy (All Routes)",
            "ruler_100_pct",
            readable_dict["agency_spatial_accuracy"]["title"],
            readable_dict["spatial_accuracy_graph"]["subtitle"],
            color_dict["spatial_accuracy_domain"],
            color_dict["spatial_accuracy_range"]
        )
    )
    

alt.LayerChart(...)